In [2]:
import pandas as pd

# 读取train数据
processed_data_path = './data/Pro_Train_Data.csv'
df_train = pd.read_csv(processed_data_path , encoding = 'utf-8')
df_train.head()

,ID,Age,Gender,Edu,Query_list
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 一起 壮观 衣服 网站 动漫 绘图...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 下面 怎么回事 麦当劳 旋风 勺子 吉林...


In [3]:
df_train_Age = df_train[['Age' , 'Query_list']]

In [4]:
df_train_Age.head()

,Age,Query_list
0,1,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,2,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,4,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,4,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 一起 壮观 衣服 网站 动漫 绘图...
4,2,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 下面 怎么回事 麦当劳 旋风 勺子 吉林...


In [5]:
from sklearn.model_selection import train_test_split

X_Age = df_train_Age['Query_list'].values.tolist()
Y_Age = df_train_Age['Age'].values.tolist()

### 完成一个文本分类器class

In [17]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
import numpy as np

class TextClassifier():
    

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

    def save_model(self, path):
        dump((self.classifier, self.vectorizer), path)

    def load_model(self, path):
        self.classifier, self.vectorizer = load(path)

In [18]:
text_classifier = TextClassifier()
    
stratifiedk_fold = StratifiedKFold(shuffle = True , n_splits = 5)
history_score = []

for train_index , test_index in stratifiedk_fold.split(np.array(X_Age) , np.array(Y_Age) ):
    X_train , X_test = np.array(X_Age)[train_index] , np.array(X_Age)[test_index]
    y_train , y_test = np.array(Y_Age)[train_index] , np.array(Y_Age)[test_index]
    text_classifier.fit(X_train, y_train)

    history_score.append(text_classifier.score(X_test, y_test))
    print('Please Waiting...')

Please Waiting...
Please Waiting...
Please Waiting...
Please Waiting...
Please Waiting...


In [19]:
print('SVM')
print('max_features = 20000')
print('------------------------')
print( history_score)

SVM
max_features = 20000
------------------------
[0.5049, 0.5094, 0.5085, 0.507, 0.50615]


#### 以上对Age标签的NB模型处理。当改变vec的参数时，可以提高性能：
|当vec是默认参数时：（即analyzer = 'word' ,max_features = 4000 ,时）--> | [0.4767, 0.47915, 0.4777, 0.4803, 0.4815]  |

|SVM: | [0.47935, 0.4759, 0.4773, 0.4758, 0.48935]  

|当vec参数为:      （analyzer = 'word' ,max_features = 20000时） --> | [0.50875, 0.50375, 0.50635, 0.50655, 0.5047]|    

| SVM:[0.5049, 0.5094, 0.5085, 0.507, 0.50615]

|当vec参数为:(ngram_range=(1,4) ,analyzer = 'word' , max_features = 20000时）-->  | [0.50145, 0.5042, 0.50475, 0.5015, 0.50295]  |